In [1]:
import pandas as pd
import json
import logging
import os
from dotenv import load_dotenv

load_dotenv()

# --- Configuration ---
EXCEL_FILE = r"C:\Users\Anshd\Downloads\ai_project_scraper\Python Gigs on fiverr.xlsx"  # Path to your Excel file
OUTPUT_FORMAT = os.getenv("OUTPUT_FORMAT", "json")  # Default output format is JSON

# --- Logging Setup ---
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")

console_handler = logging.StreamHandler()
console_handler.setFormatter(formatter)

file_handler = logging.FileHandler("excel_data_processor.log")  # Log file name
file_handler.setFormatter(formatter)

logger.addHandler(console_handler)
logger.addHandler(file_handler)

def log_error(message):
    logger.error(message)

In [6]:
class ExcelDataProcessor:
    def __init__(self, excel_file):
        self.excel_file = excel_file

    def load_and_filter_data(self):
        """Loads the Excel data and filters for AI-related gigs."""
        try:
            # Load the Excel file into a pandas DataFrame
            df = pd.read_excel(self.excel_file)

            # Clean column names: remove leading/trailing spaces
            df.columns = df.columns.str.strip()

            # Filter for rows containing "Artificial Intelligence" or "AI" (case-insensitive) in any relevant column
            ai_related_df = df[
                df.apply(lambda row: row.astype(str).str.contains("Artificial Intelligence|AI", case=False, na=False).any(), axis=1)
            ]

            # Select relevant columns and convert to a list of dictionaries
            ai_projects = ai_related_df.to_dict("records")

            return ai_projects
        except FileNotFoundError:
            log_error(f"Error: File not found at {self.excel_file}")
            return []
        except Exception as e:
            log_error(f"Error loading or filtering data: {e}")
            return []

    def clean_data(self, data):
        """Handles NaN values and performs other data cleaning."""

        if not data:  # Check if the data list is empty
            return []

        # Convert the list of dictionaries to a DataFrame
        df = pd.DataFrame(data)

        # --- Handle NaN Values ---
        # Replace NaN with appropriate values or drop rows/columns based on your needs.
        # Here's an example of how to handle NaN values in specific columns:

        # Replace NaN in 'Basic Price' with "N/A"
        df['Basic Price'] = df['Basic Price'].fillna("N/A")

        # Replace NaN in 'Total Number of Reviews' with 0 (if it makes sense contextually)
        df['Total Number of Reviews'] = df['Total Number of Reviews'].fillna(0)

        # Replace NaN in 'Ratings' with a default value or the mean/median
        # df['Ratings'] = df['Ratings'].fillna(0)  # Replace with 0
        # df['Ratings'] = df['Ratings'].fillna(df['Ratings'].mean())  # Replace with mean

        # Drop rows where 'Profile Name' or 'GIG Title' is NaN
        df.dropna(subset=['Profile Name', 'GIG Title'], inplace=True)

        # Convert remaining NaN values to None for JSON compatibility:
        df = df.replace({pd.NA: None, float('nan'): None})

       

        # Convert back to a list of dictionaries
        cleaned_data = df.to_dict("records")

        return cleaned_data

In [7]:
class DataHandler:
    def __init__(self, output_format="json", output_filename="fiverr_ai_projects"):
        self.output_format = output_format
        self.output_filename = output_filename

    def save_data(self, data):
        """Saves the processed data to a file (JSON or CSV)."""
        try:
            if self.output_format == "json":
                with open(f"{self.output_filename}.json", "w", encoding="utf-8") as f:
                    json.dump(data, f, indent=4, ensure_ascii=False)
                logger.info(f"Data successfully saved to {self.output_filename}.json")
            elif self.output_format == "csv":
                with open(f"{self.output_filename}.csv", "w", newline="", encoding="utf-8") as f:
                    writer = csv.DictWriter(f, fieldnames=data[0].keys() if data else [])
                    writer.writeheader()
                    writer.writerows(data)
                logger.info(f"Data successfully saved to {self.output_filename}.csv")
            else:
                raise ValueError(f"Unsupported output format: {self.output_format}")
        except (IOError, TypeError, ValueError) as e:
            log_error(f"Error saving data: {e}")

In [8]:
def main():
    """Main function to process the Excel data."""
    logger.info("Starting the Fiverr Excel data processor...")

    processor = ExcelDataProcessor(EXCEL_FILE)
    data_handler = DataHandler(output_format=OUTPUT_FORMAT)

    ai_projects = processor.load_and_filter_data()
    if ai_projects:
        cleaned_data = processor.clean_data(ai_projects)
        data_handler.save_data(cleaned_data)
    else:
        logger.info("No AI-related projects found in the Excel file.")

    logger.info("Excel data processing finished.")

In [9]:
if __name__ == "__main__":
    main()

2025-01-10 00:47:08,918 - __main__ - INFO - Starting the Fiverr Excel data processor...
2025-01-10 00:47:09,317 - __main__ - INFO - Data successfully saved to fiverr_ai_projects.json
2025-01-10 00:47:09,317 - __main__ - INFO - Excel data processing finished.
